In [24]:
import scipy.stats
import numpy as np
import plotly.express as px
import chart_studio.plotly as py
from plotly.offline import iplot
import toleranceinterval as ti
from functools import lru_cache

NaN = float('nan')
population=0.95
confidence=0.95
alpha=1-confidence
n_mca_samples=6
n_voxels=5000
mca_samples_range = [6, 10, 30, 100,500,1000,2500,5000]


# Mean estimator

### Unbiased estimator

$M_n = \frac{\sum{X_i}}{n}$

# Variance estimators

### Biased estimator

$S^{2}_n = \frac{\sum{(M_n-\mu)^2}}{n}$

### Unbiased estimator

$S^{2}_{n-1} = \frac{\sum{(M_n-\mu)^2}}{n-1}$

# Standard deviation estimators

### Biased estimator

$S_n = \sqrt{S^2_{n}} = \sqrt{\frac{\sum{(M_n-\mu)^2}}{n}}$

### Biased estimator 2

$S_{n-1} = \sqrt{S^2_{n-1}} = \sqrt{\frac{\sum{(M_n-\mu)^2}}{n-1}}$

### Unbiased estimator

$\hat{S}_{n-1} = c_4(n)^{-1}\sqrt{S^2_{n-1}} = c_4(n)^{-1} \sqrt{\frac{\sum{(M_n-\mu)^2}}{n-1}}$
with
$c_4(n) = \sqrt{\frac{2}{n-1}}\frac{\Gamma(n-2)}{\Gamma(\frac{n-1}{2})}$

# Prediction interval

If $X$ is normally distributed, then PI is an two-sided prediction interval with an $\alpha$ confidence level for one future observation:

$ PI = M \pm \sqrt{1 + \frac{1}{n}}.t_{1-\alpha/2,n-1}.S $

# Tolerance interval

If $X$ is normally distributed, then PI is an two-sided tolerance interval with an $\alpha$ confidence level that contains $p\%$ of the population:

$$ TI = M \pm k_2.S $$

with $k_2$ the solution of the integral equation:

$$
\sqrt{\frac{2n}{\pi}} \int_{0}^{\infty} 
\left( 1 - P_Q 
    \left( 
        Q \geq \frac{\chi^2_{1,p}(x^2)}{k^2_2}
    \right) e^{-\frac{x^2}{2S^2}dx}
\right) = 1 - \alpha 
$$



In [25]:
# Estimator for the mean

def mean(x):
    return np.mean(x)

# Estimators for the variance

def var_biased(x):
    '''
    S^2_{n} = sum((X-mu)^2)/N
    '''
    _mean = mean(x)
    _n = x.size
    _diff = np.sum(np.square(x - _mean))
    return _diff/_n

def var_unbiased(x):
    '''
    S^2_{n-1} = sum((X-mu)^2)/(N-1)
    '''
    _mean = mean(x)
    _n = x.size - 1
    _diff = np.sum(np.square(x - _mean))
    return _diff/_n

# Estimators for the standard deviation

def std_biased_1(x):
    '''
    std = sqrt( S^2_n )
    '''
    return np.sqrt(var_biased(x))

def std_biased_2(x):
    '''
    std = sqrt( S^2_{n-1} )
    '''
    return np.sqrt(var_unbiased(x))

def c4(n):
    '''
    c4(n) = sqrt(2/n-1) (gamma(n/2)/gamma(n-1/2))
    '''
    gamma = scipy.special.loggamma
    return np.sqrt(2/(n-1)) * np.exp(gamma(n/2)-gamma((n-1)/2))

def std_unbiased(x):
    '''
    std = sqrt( S^2_{n-1} ) / c4(n)
    '''
    n = x.size
    _std = std_biased_2(x)/c4(n)
    return _std

# Function to draw samples

def draw_normal_samples(n):
    x = np.random.normal(size=n)
    return x

def draw_skewnorm_samples(n):
    return np.concatenate([np.random.normal(loc=20,size=int(n/2)),scipy.stats.pareto.rvs(2,size=int(n/2))])
    return scipy.stats.pareto.rvs(1,size=n)

def draw_uniform_samples(n):
    x = np.random.uniform(-100,100,size=n)
    return x

def draw_bimodal_samples(n):
    x1 = np.random.normal(loc=-2,size=int(n/2))
    x2 = np.random.normal(loc=2,size=int(n/2))
    return np.concatenate([x1,x2])

def draw_samples(n, dist):
    if dist == 'norm':
        return draw_normal_samples(n)
    elif dist == 'skew':
        return draw_skewnorm_samples(n)
    elif dist == 'uniform':
        return draw_uniform_samples(n)
    elif dist == 'bimodal':
        return draw_bimodal_samples(n)

# Function for testing prediction interval

def is_in_prediction_interval(x, mean, std, confidence):
    '''
    Check x values are in prediction interval
    alpha = confidence
    c = sqrt(1 + 1/n)
    t = t_{1-alpha/2, n-1}
    (mean - t * c * std, mean + t * c * std )
    '''
    n = x.size
    coef_t = scipy.stats.t.ppf(1-alpha/2, df=n-1)
    c = np.sqrt(1 + 1/n)
    success = np.logical_and(mean - coef_t * c * std <= x, x <= mean + coef_t * c * std)
    return success

def get_success_prediction(x, mean, std, confidence):
    trials = x.size
    success = is_in_prediction_interval(x, mean, std, confidence)
    nb_success = np.count_nonzero(success)
    nb_fail = trials - nb_success
    return trials, nb_success, nb_fail

def test_prediction(n, std, confidence, dist):    
    x = draw_samples(n, dist)
    _mean = mean(x)
    _std = std(x)
    y = draw_samples(n, dist)
    nb_trials, nb_success, nb_fail = get_success_prediction(y, _mean, _std, confidence)
    return nb_success

def run_prediction(std, n_mca_samples, n_voxels, confidence, dist):
    np.random.seed(0)
    nb_successes = [test_prediction(n_mca_samples, std, confidence, dist) for i in range(n_voxels)]
    per_succ = np.sum(nb_successes)/(n_voxels*n_mca_samples) * 100
    return per_succ


# Functions for testing tolerance interval

def compute_bounds(n, v):
    if v % 2 == 0:
        v1, v2 = v/2, v/2
    else:
        v1 = v/2 - 1/2
        v2 = v1 + 1
    l = v1
    u = (n-1) - v2 + 1
    return int(np.floor(l)), int(np.floor(u))

@lru_cache(maxsize=1000)
def epti(n, p, confidence):
    '''
    Return the maximal lower (l) and minimal upper (u) bound satisfying:
        B(u-l-1,n,p) >= confidence
    
    Reference
    ---------
    [1] Lu, Lu. "Statistical Intervals: A Guide for Practitioners and Researchers"
        by William Q. Meeker, Gerald J. Hahn, and Luis A. Escobar. Hoboken, 
        NJ: Wiley, 2017, xxxv+ 579 pp., $87.22, ISBN: 978-0-471-68717-7." (2020): 1-285.
    '''
    fmt = int(np.ceil(np.log10(n)))    
    for i in range(1, int(n/2)):
        l, u = compute_bounds(n, i)
        alpha = scipy.stats.binom.cdf(u - l - 1, n, p)
        if alpha < confidence:
            if i == 1:
                #print(f'No bounds found, best confidence expected is {alpha}')
                return None
            return compute_bounds(n, i-1)
            
def compute_k2(n,p,c):    
    return {6: 4.422150364830485, 10: 3.3934294787126014, 30: 2.5548928132776925,
            100: 2.233882023044249, 500: 2.0702285126532165, 1000: 2.036114277875941,
            2500: 2.00714815776156, 5000: 1.9929903484319966}[n]
    return ti.twoside.normal_factor(n, population, confidence)
    
def normal_tolerance_interval(n, mean, std, population, confidence):
    k2 = compute_k2(n, population, confidence)
    return mean - k2*std, mean + k2*std

def non_parametric_tolerance_interval(x, population, confidence):
    try:
        l,u = epti(x.size, population, confidence)
        x.sort()
        return x[l],x[u]
    except TypeError:
        return None
        
def is_in_tolerance_interval(x, mean, std, population, confidence, non_parametric=False):
    '''
    Check x values are in tolerance interval:
    (mean - k_2 * std, mean + k_2 * std )
    with confidence and population.
    '''
    n = x.size
    if non_parametric:
        try:
            lower_bound, upper_bound = non_parametric_tolerance_interval(x, population, confidence)
        except TypeError:
            return None
    else:
            lower_bound, upper_bound = normal_tolerance_interval(n, mean, std, population, confidence)
    success = np.logical_and(lower_bound <= x, x <= upper_bound)
    return success

def get_success_tolerance(x, mean, std, population, confidence, non_parametric=False):
    trials = x.size
    success = is_in_tolerance_interval(x, mean, std, population, confidence, non_parametric)
    nb_success = NaN if success is None else np.count_nonzero(success)
    nb_fail = trials - nb_success
    return trials, nb_success, nb_fail

def test_tolerance(n, std, population, confidence, dist, non_parametric=False):    
    x = draw_samples(n, dist)
    _mean = mean(x)
    _std = std(x)
    y = draw_samples(n, dist)
    nb_trials, nb_success, nb_fail = get_success_tolerance(y, _mean, _std, population, confidence, non_parametric)
    return nb_success

def run_tolerance(std, n_mca_samples, n_voxels, population, confidence, dist, non_parametric=False):
    np.random.seed(0)
    nb_successes = [test_tolerance(n_mca_samples, std, population, confidence, dist, non_parametric) for i in range(n_voxels)]
    per_succ = np.sum(nb_successes)/(n_voxels*n_mca_samples) * 100
    return per_succ

# Main function

def main_prediction(n_voxels, mca_samples_range, std, confidence, dist):
    print(f'Test prediction interval with {std.__name__} for confidence ({confidence}) (dist={dist})')
    for n_mca_samples in mca_samples_range:
        ratio = run_prediction(std, n_mca_samples, n_voxels, confidence, dist)
        print(f'n_voxels: {n_voxels}, n_mca_samples: {n_mca_samples:5}, success ratio: {ratio:.2f}%')
        
def main_tolerance(n_voxels, mca_samples_range, std, population, confidence, dist, non_parametric=False):
    p = 'non parametric' if non_parametric else 'parametric'
    print(f'Test tolerance interval with {std.__name__} for confidence ({confidence}) and population ({population}) (dist={dist}) ({p})')
    for n_mca_samples in mca_samples_range:
        ratio = run_tolerance(std, n_mca_samples, n_voxels, population, confidence, dist, non_parametric)
        print(f'n_voxels: {n_voxels}, n_mca_samples: {n_mca_samples:5}, success ratio: {ratio:.2f}%')        

In [26]:
main_prediction(n_voxels, mca_samples_range, std_biased_1, confidence, 'norm')
main_prediction(n_voxels, mca_samples_range, std_biased_1, confidence, 'skew')
main_prediction(n_voxels, mca_samples_range, std_biased_1, confidence, 'uniform')
main_prediction(n_voxels, mca_samples_range, std_biased_1, confidence, 'bimodal')

Test prediction interval with std_biased_1 for confidence (0.95) (dist=skew)
n_voxels: 5000, n_mca_samples:     6, success ratio: 99.95%
n_voxels: 5000, n_mca_samples:    10, success ratio: 99.96%
n_voxels: 5000, n_mca_samples:    30, success ratio: 99.95%
n_voxels: 5000, n_mca_samples:   100, success ratio: 99.94%
n_voxels: 5000, n_mca_samples:   500, success ratio: 99.94%
n_voxels: 5000, n_mca_samples:  1000, success ratio: 99.94%
n_voxels: 5000, n_mca_samples:  2500, success ratio: 99.94%
n_voxels: 5000, n_mca_samples:  5000, success ratio: 99.94%
Test prediction interval with std_biased_1 for confidence (0.95) (dist=norm)
n_voxels: 5000, n_mca_samples:     6, success ratio: 93.62%
n_voxels: 5000, n_mca_samples:    10, success ratio: 94.01%
n_voxels: 5000, n_mca_samples:    30, success ratio: 94.63%
n_voxels: 5000, n_mca_samples:   100, success ratio: 94.91%
n_voxels: 5000, n_mca_samples:   500, success ratio: 94.99%
n_voxels: 5000, n_mca_samples:  1000, success ratio: 94.99%
n_voxe

In [27]:
main_tolerance(n_voxels, mca_samples_range, std_biased_1, population, confidence, 'norm')
main_tolerance(n_voxels, mca_samples_range, std_biased_1, population, confidence, 'skew')
main_tolerance(n_voxels, mca_samples_range, std_biased_1, population, confidence, 'uniform')
main_tolerance(n_voxels, mca_samples_range, std_biased_1, population, confidence, 'bimodal')

Test tolerance interval with std_biased_1 for confidence (0.95) and population (0.95) (dist=skew) (parametric)
n_voxels: 5000, n_mca_samples:     6, success ratio: 99.96%
n_voxels: 5000, n_mca_samples:    10, success ratio: 99.97%
n_voxels: 5000, n_mca_samples:    30, success ratio: 99.96%
n_voxels: 5000, n_mca_samples:   100, success ratio: 99.94%
n_voxels: 5000, n_mca_samples:   500, success ratio: 99.94%
n_voxels: 5000, n_mca_samples:  1000, success ratio: 99.94%
n_voxels: 5000, n_mca_samples:  2500, success ratio: 99.94%
n_voxels: 5000, n_mca_samples:  5000, success ratio: 99.94%
Test tolerance interval with std_biased_1 for confidence (0.95) and population (0.95) (dist=skew) (parametric)
n_voxels: 5000, n_mca_samples:     6, success ratio: 99.96%
n_voxels: 5000, n_mca_samples:    10, success ratio: 99.97%
n_voxels: 5000, n_mca_samples:    30, success ratio: 99.96%
n_voxels: 5000, n_mca_samples:   100, success ratio: 99.94%
n_voxels: 5000, n_mca_samples:   500, success ratio: 99.94

In [28]:
main_tolerance(n_voxels, mca_samples_range, std_biased_1, population, confidence, 'norm', non_parametric=True)
main_tolerance(n_voxels, mca_samples_range, std_biased_1, population, confidence, 'skew', non_parametric=True)
main_tolerance(n_voxels, mca_samples_range, std_biased_1, population, confidence, 'uniform', non_parametric=True)
main_tolerance(n_voxels, mca_samples_range, std_biased_1, population, confidence, 'bimodal', non_parametric=True)


Test tolerance interval with std_biased_1 for confidence (0.95) and population (0.95) (dist=norm) (non parametric)
n_voxels: 5000, n_mca_samples:     6, success ratio: nan%
n_voxels: 5000, n_mca_samples:    10, success ratio: nan%
n_voxels: 5000, n_mca_samples:    30, success ratio: nan%
n_voxels: 5000, n_mca_samples:   100, success ratio: 100.00%
n_voxels: 5000, n_mca_samples:   500, success ratio: 97.00%
n_voxels: 5000, n_mca_samples:  1000, success ratio: 96.30%
n_voxels: 5000, n_mca_samples:  2500, success ratio: 95.80%
n_voxels: 5000, n_mca_samples:  5000, success ratio: 95.54%
Test tolerance interval with std_biased_1 for confidence (0.95) and population (0.95) (dist=skew) (non parametric)
n_voxels: 5000, n_mca_samples:     6, success ratio: nan%
n_voxels: 5000, n_mca_samples:    10, success ratio: nan%
n_voxels: 5000, n_mca_samples:    30, success ratio: nan%
n_voxels: 5000, n_mca_samples:   100, success ratio: 100.00%
n_voxels: 5000, n_mca_samples:   500, success ratio: 97.00%


In [29]:
main_prediction(n_voxels, mca_samples_range, std_biased_2, confidence, 'norm')
main_prediction(n_voxels, mca_samples_range, std_biased_2, confidence, 'skew')
main_prediction(n_voxels, mca_samples_range, std_biased_2, confidence, 'uniform')
main_prediction(n_voxels, mca_samples_range, std_biased_2, confidence, 'bimodal')


Test prediction interval with std_biased_2 for confidence (0.95) (dist=skew)
n_voxels: 5000, n_mca_samples:     6, success ratio: 99.95%
n_voxels: 5000, n_mca_samples:    10, success ratio: 99.96%
n_voxels: 5000, n_mca_samples:    30, success ratio: 99.95%
n_voxels: 5000, n_mca_samples:   100, success ratio: 99.94%
n_voxels: 5000, n_mca_samples:   500, success ratio: 99.94%
n_voxels: 5000, n_mca_samples:  1000, success ratio: 99.94%
n_voxels: 5000, n_mca_samples:  2500, success ratio: 99.94%
n_voxels: 5000, n_mca_samples:  5000, success ratio: 99.94%
Test prediction interval with std_biased_2 for confidence (0.95) (dist=norm)
n_voxels: 5000, n_mca_samples:     6, success ratio: 95.12%
n_voxels: 5000, n_mca_samples:    10, success ratio: 95.01%
n_voxels: 5000, n_mca_samples:    30, success ratio: 95.00%
n_voxels: 5000, n_mca_samples:   100, success ratio: 95.02%
n_voxels: 5000, n_mca_samples:   500, success ratio: 95.01%
n_voxels: 5000, n_mca_samples:  1000, success ratio: 95.01%
n_voxe

In [30]:
main_tolerance(n_voxels, mca_samples_range, std_biased_2, population, confidence, 'norm')
main_tolerance(n_voxels, mca_samples_range, std_biased_2, population, confidence, 'skew')
main_tolerance(n_voxels, mca_samples_range, std_biased_2, population, confidence, 'uniform')
main_tolerance(n_voxels, mca_samples_range, std_biased_2, population, confidence, 'bimodal')

Test tolerance interval with std_biased_2 for confidence (0.95) and population (0.95) (dist=skew) (parametric)
n_voxels: 5000, n_mca_samples:     6, success ratio: 99.97%
n_voxels: 5000, n_mca_samples:    10, success ratio: 99.97%
n_voxels: 5000, n_mca_samples:    30, success ratio: 99.96%
n_voxels: 5000, n_mca_samples:   100, success ratio: 99.95%
n_voxels: 5000, n_mca_samples:   500, success ratio: 99.94%
n_voxels: 5000, n_mca_samples:  1000, success ratio: 99.94%
n_voxels: 5000, n_mca_samples:  2500, success ratio: 99.94%
n_voxels: 5000, n_mca_samples:  5000, success ratio: 99.94%
Test tolerance interval with std_biased_2 for confidence (0.95) and population (0.95) (dist=norm) (parametric)
n_voxels: 5000, n_mca_samples:     6, success ratio: 99.19%
n_voxels: 5000, n_mca_samples:    10, success ratio: 99.02%
n_voxels: 5000, n_mca_samples:    30, success ratio: 98.21%
n_voxels: 5000, n_mca_samples:   100, success ratio: 97.16%
n_voxels: 5000, n_mca_samples:   500, success ratio: 96.10

In [31]:
main_tolerance(n_voxels, mca_samples_range, std_biased_2, population, confidence, 'norm', non_parametric=True)
main_tolerance(n_voxels, mca_samples_range, std_biased_2, population, confidence, 'skew', non_parametric=True)
main_tolerance(n_voxels, mca_samples_range, std_biased_2, population, confidence, 'uniform', non_parametric=True)
main_tolerance(n_voxels, mca_samples_range, std_biased_2, population, confidence, 'bimodal', non_parametric=True)

Test tolerance interval with std_biased_2 for confidence (0.95) and population (0.95) (dist=skew) (non parametric)
n_voxels: 5000, n_mca_samples:     6, success ratio: nan%
n_voxels: 5000, n_mca_samples:    10, success ratio: nan%
n_voxels: 5000, n_mca_samples:    30, success ratio: nan%
n_voxels: 5000, n_mca_samples:   100, success ratio: 100.00%
n_voxels: 5000, n_mca_samples:   500, success ratio: 97.00%
n_voxels: 5000, n_mca_samples:  1000, success ratio: 96.30%
n_voxels: 5000, n_mca_samples:  2500, success ratio: 95.80%
n_voxels: 5000, n_mca_samples:  5000, success ratio: 95.54%
Test tolerance interval with std_biased_2 for confidence (0.95) and population (0.95) (dist=norm) (non parametric)
n_voxels: 5000, n_mca_samples:     6, success ratio: nan%
n_voxels: 5000, n_mca_samples:    10, success ratio: nan%
n_voxels: 5000, n_mca_samples:    30, success ratio: nan%
n_voxels: 5000, n_mca_samples:   100, success ratio: 100.00%
n_voxels: 5000, n_mca_samples:   500, success ratio: 97.00%


In [32]:
main_prediction(n_voxels, mca_samples_range, std_unbiased, confidence, 'norm')
main_prediction(n_voxels, mca_samples_range, std_unbiased, confidence, 'skew')
main_prediction(n_voxels, mca_samples_range, std_unbiased, confidence, 'uniform')
main_prediction(n_voxels, mca_samples_range, std_unbiased, confidence, 'bimodal')

Test prediction interval with std_unbiased for confidence (0.95) (dist=norm)
n_voxels: 5000, n_mca_samples:     6, success ratio: 95.88%
n_voxels: 5000, n_mca_samples:    10, success ratio: 95.53%
n_voxels: 5000, n_mca_samples:    30, success ratio: 95.17%
n_voxels: 5000, n_mca_samples:   100, success ratio: 95.07%
n_voxels: 5000, n_mca_samples:   500, success ratio: 95.02%
n_voxels: 5000, n_mca_samples:  1000, success ratio: 95.01%
n_voxels: 5000, n_mca_samples:  2500, success ratio: 95.01%
n_voxels: 5000, n_mca_samples:  5000, success ratio: 94.99%
Test prediction interval with std_unbiased for confidence (0.95) (dist=skew)
n_voxels: 5000, n_mca_samples:     6, success ratio: 99.95%
n_voxels: 5000, n_mca_samples:    10, success ratio: 99.96%
n_voxels: 5000, n_mca_samples:    30, success ratio: 99.95%
n_voxels: 5000, n_mca_samples:   100, success ratio: 99.94%
n_voxels: 5000, n_mca_samples:   500, success ratio: 99.94%
n_voxels: 5000, n_mca_samples:  1000, success ratio: 99.94%
n_voxe

In [33]:
main_tolerance(n_voxels, mca_samples_range, std_unbiased, population, confidence, 'norm')    
main_tolerance(n_voxels, mca_samples_range, std_unbiased, population, confidence, 'skew')    
main_tolerance(n_voxels, mca_samples_range, std_unbiased, population, confidence, 'uniform')    
main_tolerance(n_voxels, mca_samples_range, std_unbiased, population, confidence, 'bimodal')    

Test tolerance interval with std_unbiased for confidence (0.95) and population (0.95) (dist=norm) (parametric)
n_voxels: 5000, n_mca_samples:     6, success ratio: 99.30%
n_voxels: 5000, n_mca_samples:    10, success ratio: 99.14%
n_voxels: 5000, n_mca_samples:    30, success ratio: 98.29%
n_voxels: 5000, n_mca_samples:   100, success ratio: 97.19%
n_voxels: 5000, n_mca_samples:   500, success ratio: 96.11%
n_voxels: 5000, n_mca_samples:  1000, success ratio: 95.80%
n_voxels: 5000, n_mca_samples:  2500, success ratio: 95.52%
n_voxels: 5000, n_mca_samples:  5000, success ratio: 95.36%
Test tolerance interval with std_unbiased for confidence (0.95) and population (0.95) (dist=skew) (parametric)
n_voxels: 5000, n_mca_samples:     6, success ratio: 99.97%
n_voxels: 5000, n_mca_samples:    10, success ratio: 99.97%
n_voxels: 5000, n_mca_samples:    30, success ratio: 99.96%
n_voxels: 5000, n_mca_samples:   100, success ratio: 99.95%
n_voxels: 5000, n_mca_samples:   500, success ratio: 99.94

In [34]:
main_tolerance(n_voxels, mca_samples_range, std_unbiased, population, confidence, 'norm', non_parametric=True)    
main_tolerance(n_voxels, mca_samples_range, std_unbiased, population, confidence, 'skew', non_parametric=True)    
main_tolerance(n_voxels, mca_samples_range, std_unbiased, population, confidence, 'uniform', non_parametric=True)    
main_tolerance(n_voxels, mca_samples_range, std_unbiased, population, confidence, 'bimodal', non_parametric=True)    

Test tolerance interval with std_unbiased for confidence (0.95) and population (0.95) (dist=norm) (non parametric)
n_voxels: 5000, n_mca_samples:     6, success ratio: nan%
n_voxels: 5000, n_mca_samples:    10, success ratio: nan%
n_voxels: 5000, n_mca_samples:    30, success ratio: nan%
n_voxels: 5000, n_mca_samples:   100, success ratio: 100.00%
n_voxels: 5000, n_mca_samples:   500, success ratio: 97.00%
n_voxels: 5000, n_mca_samples:  1000, success ratio: 96.30%
n_voxels: 5000, n_mca_samples:  2500, success ratio: 95.80%
n_voxels: 5000, n_mca_samples:  5000, success ratio: 95.54%
Test tolerance interval with std_unbiased for confidence (0.95) and population (0.95) (dist=skew) (non parametric)
n_voxels: 5000, n_mca_samples:     6, success ratio: nan%
n_voxels: 5000, n_mca_samples:    10, success ratio: nan%
n_voxels: 5000, n_mca_samples:    30, success ratio: nan%
n_voxels: 5000, n_mca_samples:   100, success ratio: 100.00%
n_voxels: 5000, n_mca_samples:   500, success ratio: 97.00%
